# TAREA. Dataset logs nasa

### ETL

Start Date: 29/11/2019

End Date: 29/11/2019

### José María Álvarez Silva

El objetivo de esta práctica es emplear el RDD access_logs para realizar una serie de cálculos:

 * Se ha realizado con `pyspark`
 * Ahora se hará con `pyspark sql`

## 1. En primer lugar se deben cargar los datos en un contexto Spark:

In [17]:
from pyspark import SparkContext
from datetime import datetime
import pandas as pd

In [2]:
sc = SparkContext()

In [3]:
data_file = "./apache.access.log"
raw_data = sc.textFile(data_file)

Hasta aquí se han cargado los datos en el RDD raw_data. Como primera medida para asegurar que los datos se han subido de manera correcta se procede al conteo de los mismos:

In [4]:
raw_data.count()

3432

Además vamos a visualizar los primero elementos del RDD:

In [5]:
raw_data.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

Una vez confirmado que la carga ha sido realizada de manera correcta se procede al parseado de los datos:

En el parseado se emplea la expresión regular vista hoy en clase:

In [6]:
import re
def parse_log1(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([\w\.\s*]+)?\s*"*(\d{3}) (\S+)', line)
    if match is None:
        return 0
    else:
        return 1
n_logs = raw_data.count()

In [7]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)


In [8]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])
parsed_def = parsed_rdd.map(lambda line: map_log(line))

In [9]:
parsed_rdd.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

In [10]:
parsed_def.take(2)

[('in24.inetnebr.com',
  '-',
  '-',
  '01/Aug/1995:00:00:01',
  '0400',
  'GET',
  '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
  'HTTP/1.0"',
  None,
  '200',
  '1839'),
 ('uplherc.upl.com',
  '-',
  '-',
  '01/Aug/1995:00:00:07',
  '0400',
  'GET',
  '/',
  'HTTP/1.0"',
  None,
  '304',
  '0')]

## Pregunta 1

## Mínimo, máximo y media del tamaño de las peticiones

A continuación se presentan las estadísticas (incluyendo mínimo, máximo y media) del tamaño de las peticiones:

In [11]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)
parsed_def.map(lambda line: convert_long(line[-1])).stats()

(count: 3432, mean: 16051.863636363621, stdev: 53247.8157482, max: 887988.0, min: 0.0)

In [12]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import Row

In [20]:
row_data = parsed_def.map(lambda p: Row(
    host = p[0], 
#     fecha = (datetime.strptime(p[3][:11], "%d/%b%Y")),
    endpoint = p[6],
    size = convert_long(p[-1]),
    repuesta = p[-2]
    )
)

In [21]:
lognasa_df = sqlContext.createDataFrame(row_data)
lognasa_df.registerTempTable("lognasa")

In [24]:
statsNASA = sqlContext.sql("""
    SELECT MIN(size), MAX(size), MEAN(size) FROM lognasa
""")
statsNASA.show()

+---------+---------+------------------+
|min(size)|max(size)|         avg(size)|
+---------+---------+------------------+
|        0|   887988|16051.863636363636|
+---------+---------+------------------+



## Pregunta 2

## Número de peticiones de cada código de respuesta

El siguiente objetivo es calcular el número de peticiones de cada código de respuesta:

In [ ]:
n_codes = parsed_def.map(lambda line: (line[-2], 1)).distinct().count()
codes_count = (parsed_def.map(lambda line: (line[-2], 1))
          .reduceByKey(lambda a, b: a + b)
          .takeOrdered(n_codes, lambda x: -x[1]))
codes_count

In [29]:
respuestasCode = sqlContext.sql("""
    SELECT repuesta, COUNT(repuesta) FROM lognasa GROUP BY repuesta
""")
respuestasCode.show()

+--------+---------------+
|repuesta|count(repuesta)|
+--------+---------------+
|     200|           3140|
|     302|             50|
|     404|             22|
|     403|              1|
|     304|            219|
+--------+---------------+



## Pregunta 3

## Mostrar 20 hosts que han sido visitados más de 10 veces

Si se desean ver  20 hosts visitados más de diez veces:

In [33]:
result = parsed_def.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b).filter(lambda x : x[1] > 10).takeOrdered(20, lambda x: -x[1])
result

[('ix-min1-02.ix.netcom.com', 78),
 ('uplherc.upl.com', 71),
 ('port26.ts1.msstate.edu', 59),
 ('h96-158.ccnet.com', 56),
 ('in24.inetnebr.com', 55),
 ('piweba3y.prodigy.com', 54),
 ('thing1.cchem.berkeley.edu', 54),
 ('adam.tower.com.au', 44),
 ('ip55.van2.pacifier.com', 43),
 ('ppp1016.po.iijnet.or.jp', 41),
 ('hsccs_gatorbox07.unm.edu', 40),
 ('www-b2.proxy.aol.com', 40),
 ('www-d1.proxy.aol.com', 39),
 ('133.43.96.45', 37),
 ('port13.wavenet.com', 37),
 ('pc-heh.icl.dk', 33),
 ('haraway.ucet.ufl.edu', 32),
 ('193.84.66.147', 31),
 ('www-c1.proxy.aol.com', 30),
 ('143.158.26.50', 29)]

In [41]:
host20 = sqlContext.sql("""
   SELECT host, num FROM (SELECT host, COUNT(host) as num FROM lognasa GROUP BY host) WHERE num > 10 ORDER BY num DESC LIMIT 20
""")
host20.show()

+--------------------+---+
|                host|num|
+--------------------+---+
|ix-min1-02.ix.net...| 78|
|     uplherc.upl.com| 71|
|port26.ts1.msstat...| 59|
|   h96-158.ccnet.com| 56|
|   in24.inetnebr.com| 55|
|piweba3y.prodigy.com| 54|
|thing1.cchem.berk...| 54|
|   adam.tower.com.au| 44|
|ip55.van2.pacifie...| 43|
|ppp1016.po.iijnet...| 41|
|hsccs_gatorbox07....| 40|
|www-b2.proxy.aol.com| 40|
|www-d1.proxy.aol.com| 39|
|        133.43.96.45| 37|
|  port13.wavenet.com| 37|
|       pc-heh.icl.dk| 33|
|haraway.ucet.ufl.edu| 32|
|       193.84.66.147| 31|
|www-c1.proxy.aol.com| 30|
|term1-24.sb.west.net| 29|
+--------------------+---+



## Pregunta 4 

## Mostrar los 10 endpoints más visitados

A continuación se muestran los 10 endpoints más visitados:

In [42]:
result = parsed_def.map(lambda line: (line[6],1)).reduceByKey(lambda a, b: a + b).takeOrdered(10, lambda x: -x[1])
result

[('/images/KSC-logosmall.gif', 167),
 ('/images/NASA-logosmall.gif', 160),
 ('/images/MOSAIC-logosmall.gif', 122),
 ('/images/WORLD-logosmall.gif', 120),
 ('/images/USA-logosmall.gif', 118),
 ('/images/ksclogo-medium.gif', 106),
 ('/', 85),
 ('/history/apollo/images/apollo-logo1.gif', 74),
 ('/images/launch-logo.gif', 69),
 ('/images/ksclogosmall.gif', 66)]

In [46]:
endpoints10 = sqlContext.sql("""
   SELECT endpoint, COUNT(endpoint) as num FROM lognasa GROUP BY endpoint ORDER BY num DESC LIMIT 10
""")
endpoints10.show()

+--------------------+---+
|            endpoint|num|
+--------------------+---+
|/images/KSC-logos...|167|
|/images/NASA-logo...|160|
|/images/MOSAIC-lo...|122|
|/images/WORLD-log...|120|
|/images/USA-logos...|118|
|/images/ksclogo-m...|106|
|                   /| 85|
|/history/apollo/i...| 74|
|/images/launch-lo...| 69|
|/images/ksclogosm...| 66|
+--------------------+---+



## Pregunta 5

## Mostrar los 10 endpoints mas visitados que no tienen el codigo de respuesta 200

Si se desean los 10 endpoints más visitados que no han devuelto un resultado (es decir, que tienen un código distinto de 200):

In [47]:
result = (parsed_def.filter(lambda line: line[9] != '200')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b)
          .takeOrdered(10, lambda x: -x[1]))
result

[('/images/NASA-logosmall.gif', 25),
 ('/images/KSC-logosmall.gif', 24),
 ('/images/MOSAIC-logosmall.gif', 17),
 ('/images/WORLD-logosmall.gif', 17),
 ('/images/USA-logosmall.gif', 16),
 ('/images/ksclogo-medium.gif', 10),
 ('/software/winvn/bluemarb.gif', 8),
 ('/software/winvn/winvn.html', 8),
 ('/images/construct.gif', 8),
 ('/software/winvn/wvsmall.gif', 6)]

In [54]:
endpoints10sin200 = sqlContext.sql("""
   SELECT endpoint, COUNT(endpoint) as num FROM (SELECT endpoint FROM lognasa WHERE repuesta != 200) GROUP BY endpoint ORDER BY num DESC LIMIT 10 
""")
endpoints10sin200.show()

+--------------------+---+
|            endpoint|num|
+--------------------+---+
|/images/NASA-logo...| 25|
|/images/KSC-logos...| 24|
|/images/MOSAIC-lo...| 17|
|/images/WORLD-log...| 17|
|/images/USA-logos...| 16|
|/images/ksclogo-m...| 10|
|/software/winvn/b...|  8|
|/images/construct...|  8|
|/software/winvn/w...|  8|
|/software/winvn/w...|  6|
+--------------------+---+



## Pregunta 6

## Número de Hosts Distintos

A continuación se calcula el número de hosts distintos:

In [55]:
parsed_def.map(lambda line: line[0]).distinct().count()

311

In [57]:
distinctHosts = sqlContext.sql("""
   SELECT COUNT(DISTINCT host) FROM lognasa 
""")
distinctHosts.show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|                 311|
+--------------------+



## Pregunta 7

## Número de Hosts unicos por día

Tras ello se puede calcular el número de hosts únicos cada día:

In [58]:
def day_month(line):
    date_time = line[3]
    return datetime.strptime(date_time[:11], "%d/%b/%Y") #Se parsea la fecha para trabajar con ella tal y como vimos en clase.
result = parsed_def.map(lambda line:  (day_month(line), 1)).reduceByKey(lambda a, b: a + b).distinct().collect()
result



[(datetime.datetime(1995, 8, 1, 0, 0), 3432)]

In [68]:
row_data = parsed_def.map(lambda p: Row(
    host = p[0], 
    fecha = p[3][:11],
    endpoint = p[6],
    size = convert_long(p[-1]),
    repuesta = p[-2]
    )
)
row_data

PythonRDD[179] at RDD at PythonRDD.scala:48

In [69]:
lognasa_df = sqlContext.createDataFrame(row_data)
lognasa_df.registerTempTable("lognasa")

In [70]:
distinctHosts = sqlContext.sql("""
   SELECT fecha, COUNT(DISTINCT host) FROM lognasa GROUP BY fecha
""")
distinctHosts.show()

+-----------+--------------------+
|      fecha|count(DISTINCT host)|
+-----------+--------------------+
|01/Aug/1995|                 311|
+-----------+--------------------+



__Nota:__ En este dataset solo se poseén datos de un día por esa razón se obtiene este resultado.

## Pregunta 9 

## media de peticiones diarias por host

Tras esto se pide calcular la media de peticiones diaria por host.

In [71]:
unique_result = (parsed_def.map(lambda line:  (day_month(line),line[0]))
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1]))))

length_result = (parsed_def.map(lambda line:  (day_month(line),line[0]))
          .groupByKey().mapValues(len))

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}
day_count_df = pd.DataFrame(day_count_dct )


El panda en este caso no se usa para realizar ningún cálculo si no simplemente para una mejor visualización.

__Solución:__ 

In [72]:
day_count_df

,Día,Media
0,1995-08-01,11.03537


In [81]:
sizeFecha = sqlContext.sql("""
   SELECT fecha, MEAN(size) FROM lognasa GROUP BY fecha
""")
sizeFecha.show()

+-----------+------------------+
|      fecha|         avg(size)|
+-----------+------------------+
|01/Aug/1995|16051.863636363636|
+-----------+------------------+



In [92]:
sizeFecha = sqlContext.sql("""
   SELECT host, COUNT(DISTInCT fecha) as nfecha FROM lognasa GROUP BY host
""")
sizeFecha.show()

+--------------------+------+
|                host|nfecha|
+--------------------+------+
|ix-sea6-23.ix.net...|     1|
|grimnet23.idirect...|     1|
|        hella.stm.it|     1|
|ix-sd6-29.ix.netc...|     1|
|       198.161.85.36|     1|
|engei.engei-hs.oy...|     1|
|     info.telenor.no|     1|
|dd08-029.compuser...|     1|
|in2pc2.med.niigat...|     1|
|  server.indo.net.id|     1|
|ip-pdx4-15.telepo...|     1|
|ix-sf10-28.ix.net...|     1|
|   in24.inetnebr.com|     1|
|     204.199.188.113|     1|
|  pwestec.sierra.net|     1|
|roseanne06.slip.y...|     1|
|        206.24.43.11|     1|
|empedocles.cfar.u...|     1|
|     pm6a3.sover.net|     1|
|mcnaught.chem.su....|     1|
+--------------------+------+
only showing top 20 rows



In [93]:
sizeHost = sqlContext.sql("""
   SELECT host, COUNT(host) as nhost FROM lognasa GROUP BY host
""")
sizeHost.show()

+--------------------+-----+
|                host|nhost|
+--------------------+-----+
|ix-sea6-23.ix.net...|    9|
|grimnet23.idirect...|   10|
|        hella.stm.it|   10|
|       198.161.85.36|    1|
|ix-sd6-29.ix.netc...|    6|
|     info.telenor.no|   15|
|dd08-029.compuser...|    5|
|engei.engei-hs.oy...|   10|
|  server.indo.net.id|    1|
|in2pc2.med.niigat...|   14|
|ix-sf10-28.ix.net...|   12|
|ip-pdx4-15.telepo...|    4|
|   in24.inetnebr.com|   55|
|  pwestec.sierra.net|    8|
|roseanne06.slip.y...|    4|
|     204.199.188.113|   12|
|        206.24.43.11|    6|
|empedocles.cfar.u...|    1|
|     pm6a3.sover.net|   11|
|www-b5.proxy.aol.com|   28|
+--------------------+-----+
only showing top 20 rows



In [101]:
meanHostByFecha = sqlContext.sql("""
   Select host, nhost/nfecha as media_Peticiones_Diarias  FROM (SELECT host, COUNT(host) as nhost FROM lognasa GROUP BY host) JOIN (SELECT host as fhost, COUNT(DISTInCT fecha) as nfecha FROM lognasa GROUP BY host) ON fhost = host 
""")
meanHostByFecha.show()

+--------------------+------------------------+
|                host|media_Peticiones_Diarias|
+--------------------+------------------------+
|grimnet23.idirect...|                    10.0|
|ix-sea6-23.ix.net...|                     9.0|
|        hella.stm.it|                    10.0|
|       198.161.85.36|                     1.0|
|ix-sd6-29.ix.netc...|                     6.0|
|dd08-029.compuser...|                     5.0|
|engei.engei-hs.oy...|                    10.0|
|     info.telenor.no|                    15.0|
|in2pc2.med.niigat...|                    14.0|
|  server.indo.net.id|                     1.0|
|ip-pdx4-15.telepo...|                     4.0|
|ix-sf10-28.ix.net...|                    12.0|
|     204.199.188.113|                    12.0|
|   in24.inetnebr.com|                    55.0|
|  pwestec.sierra.net|                     8.0|
|roseanne06.slip.y...|                     4.0|
|        206.24.43.11|                     6.0|
|empedocles.cfar.u...|                  

## Pregunta 10

## se muestra una lista de 40 endpoints que generan código de respuesta = 404

A continuación, se muestra una lista de 40 endpoints que generan código de respuesta = 404:

In [84]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(40, lambda x: -x[1]))
result

[('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/readme.txt', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-004/movies/', 1),
 ('/history/history.htm', 1),
 ('/www/software/winvn/winvn.html', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1)]

__Nota:__ En la lista solo aparecen 13 endpoints porque se está trabajando con la base de datos reducida.

In [85]:
codigo404 = sqlContext.sql("""
   SELECT DISTINCT(endpoint) FROM lognasa WHERE repuesta = "404"
""")
codigo404.show()

+--------------------+
|            endpoint|
+--------------------+
|/pub/winvn/readme...|
|/elv/DELTA/uncons...|
|/history/apollo/a...|
|/history/apollo/a...|
|     /sts-71/launch/|
|/history/apollo/a...|
|/history/apollo/a...|
|/shuttle/resource...|
|/www/software/win...|
|/pub/winvn/releas...|
|/history/apollo/a...|
|/history/apollo/a...|
|/history/history.htm|
+--------------------+



## Pregunta 11

## mostrar los 25 endpoints que más código 404 generan

El siguiente comando mostraría el top 25 de endpoints que más códigos 404 generan de dispnerse de más de 13:

In [102]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(25, lambda x: -x[1]))
result

[('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/readme.txt', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-004/movies/', 1),
 ('/history/history.htm', 1),
 ('/www/software/winvn/winvn.html', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1)]

In [106]:
topCodigo404 = sqlContext.sql("""
   SELECT endpoint, COUNT(endpoint) as top FROM lognasa WHERE repuesta = "404" GROUP BY endpoint ORDER BY top DESC LIMIT 25
""")
topCodigo404.show()

+--------------------+---+
|            endpoint|top|
+--------------------+---+
|/pub/winvn/releas...|  4|
|/history/apollo/a...|  4|
|/history/apollo/a...|  2|
|/history/apollo/a...|  2|
|/pub/winvn/readme...|  2|
|/elv/DELTA/uncons...|  1|
|     /sts-71/launch/|  1|
|/history/apollo/a...|  1|
|/shuttle/resource...|  1|
|/history/apollo/a...|  1|
|/history/apollo/a...|  1|
|/www/software/win...|  1|
|/history/history.htm|  1|
+--------------------+---+



## Pregunta 12

## TOP 5 días con mayor número de 404's

Si se desean obtener el top 5 de días que más códigos de error 404 generan:

In [107]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line:  (day_month(line), 1))
          .reduceByKey(lambda a, b: a+b).collect())
day = [x[0] for x in result]
count = [x[1] for x in result]
day_count_dct = {'day':day, 'count':count}
day_count_df = pd.DataFrame(day_count_dct )


De nuevo el paso a DF se emplea únicamente para una mejor visualización:

In [108]:
day_count_df.sort_values('count', ascending = False)[:10]


,count,day
0,22,1995-08-01


__Nota:__ De nuevo se obtiene un solo día por trabajarse con la base de datos reducida.

In [109]:
top5 = sqlContext.sql("""
   SELECT fecha, COUNT(fecha) as top FROM lognasa WHERE repuesta = "404" GROUP BY fecha ORDER BY top DESC LIMIT 5
""")
top5.show()

+-----------+---+
|      fecha|top|
+-----------+---+
|01/Aug/1995| 22|
+-----------+---+



## SQL Spark

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
from pyspark.sql import Row

HOST, FECHA, ENDPOINT, TAMAÑO y ERROR

In [ ]:
row_data = parsed_def.map(lambda p: Row(
    host = p[0], 
    fecha = (datetime.strptime(p[3][:11], "%d/%b%Y")),
    endpoint = p[6],
    size = convert_long(p[-1]),
    repuesta = p[-2]
    )
)

In [ ]:
row_data.take(5)

In [ ]:
lognasa_df = sqlContext.createDataFrame(row_data)
lognasa_df.registerTempTable("lognasa")

In [ ]:
sc.stop()